In [ ]:
from copy import deepcopy
from collections import deque

import pandas as pd
import numpy as np

In [ ]:
def generate_files(disk_map: str):
    """Turn `12345` into `0..111....22222`"""
    disk_map = list(map(int, disk_map))
    files = []
    file_id = 0
    for i, n in enumerate(disk_map):
        if not i % 2:
            files.extend([file_id] * n)
            file_id += 1
        else:
            files.extend([None] * n)
    return files

In [ ]:
files = deque(generate_files("2333133121414131402"))

In [ ]:
def swap_files(files: list):
    files = deepcopy(files)
    files_dq: deque = deque(files)
    result = 0
    for i, file_id in enumerate(files):
        if i >= len(files_dq):
            break
        if file_id is not None:
            # print(f"{i}*{file_id}")
            # print(files_dq)
            result += i * file_id
            continue
        while True:
            last_file = files_dq.pop()
            if last_file is not None:
                break
        if i >= len(files_dq):
            break
        files_dq[i] = last_file
        # print(f"{i}*{last_file}")
        # print(files_dq)
        result += i * last_file
    return (
        result,
        # files_dq,
    )

In [ ]:
# file = "example"
file = "input"

In [ ]:
with open(file) as f:
    disk_map = f.read().strip()

In [ ]:
files = generate_files(disk_map)

In [ ]:
swap_files(files)

In [ ]:
# def move_files_in_blocks(files):
#     files = deepcopy(files)
#     files_dq: deque = deque(files)
#     id = 0
#     result = 0
#     while True:
#         # First, find the last block of files.
#         last_file_block = []
#         # We know that at the beginning, the last file is not None.
#         while files_dq:
#             last_file = files_dq[-1]
#             if last_file is None:
#                 if last_file_block:
#                     # It must mean that the last block is over.
#                     break
#                 # Skip the Nones at the end.
#                 files_dq.pop()
#                 continue
#             if last_file_block and last_file != last_file_block[-1]:
#                 # It must mean that the last block is over.
#                 break
#             last_file_block.append(last_file)
#             files_dq.pop()

#         # Backup the deque and result at this point in case the move fails.
#         files_dq_copy = deepcopy(files_dq)
#         result_copy = result
#         # Success flag for moving the last block.
#         succeed = False

#         # Then, find the first block of Nones that can hold the last block.
#         first_none_block = []
#         while files_dq:
#             if len(first_none_block) == len(last_file_block):
#                 # The Nones in the beginning are big enough to hold the last
#                 # block. Calculate the result and move on to the next loop.
#                 i = id - 1
#                 for file in last_file_block:
#                     result += i * file
#                     i -= 1
#                 succeed = True
#                 break

#             first_file = files_dq[0]
#             if first_file is not None:
#                 if first_none_block:
#                     # It must mean that the Nones in the beginning are not big
#                     # enough to hold the last block. Reset the first_none_block.
#                     first_none_block = []
#                 # Calculate the result since it is not None.
#                 result += id * first_file
#                 files_dq.popleft()
#                 id += 1
#                 continue

#             # It is a None. Add it to the first_none_block.
#             first_none_block.append(first_file)
#             files_dq.popleft()
#             id += 1

#         if not succeed:
#             # The last block cannot be moved. Restore the deque and result.
#             files_dq = files_dq_copy
#             result = result_copy

#     return (
#         result,
#         files_dq,
#     )




In [ ]:
def move_files_in_blocks(files: pd.Series):
    files = deepcopy(files)
    last_block_id = files.iloc[-1]
    i = len(files) - 1
    while last_block_id != 0:
        # First, find the last block of files.
        last_file_block = []
        # We know that at the beginning, the last file is not None.
        while True:
            last_file = files[i]
            if np.isnan(last_file):
                if last_file_block:
                    # It must mean that the last block is over.
                    break
                # Skip the Nones at the end.
                i -= 1
                continue
            if last_file_block and last_file != last_file_block[-1]:
                # It must mean that the last block is over.
                break
            if last_file != last_block_id:
                i -= 1
                continue
            last_file_block.append(last_file)
            i -= 1

        last_block_id = last_file_block[-1] - 1

        # Then, find the first block of Nones that can hold the last block.
        first_none_block = []
        j = 0
        while j < i:
            if len(first_none_block) == len(last_file_block):
                # The Nones in the beginning are big enough to hold the last
                # block. Calculate the result and move on to the next loop.
                print(f"Swaping {last_file_block} with {first_none_block}")
                files[j-len(last_file_block): j] = last_file_block
                files[i+1: i+1+len(last_file_block)] = first_none_block
                break

            first_file = files[j]
            if not np.isnan(first_file):
                if first_none_block:
                    # It must mean that the Nones in the beginning are not big
                    # enough to hold the last block. Reset the first_none_block.
                    first_none_block = []
                j += 1
                continue

            # It is a None. Add it to the first_none_block.
            first_none_block.append(first_file)
            j += 1

    return files


In [ ]:
def calculate_result(files: pd.Series):
    result = 0
    for i, file in enumerate(files):
        if not np.isnan(file):
            result += i * file
    return result

In [ ]:
result_files = move_files_in_blocks(pd.Series(files))

In [ ]:
print(result_files.to_list())

In [ ]:
calculate_result(result_files)

In [ ]:
# Copy other's solution

from itertools import zip_longest
from collections import deque


def parse_input(path: str) -> str:
    with open(path, "r") as input:
        return input.read().strip()


def solve(input: str) -> int:
    # (ID, (block, free))
    files = list(
        enumerate(
            zip_longest(
                list(map(int, input[::2])), list(map(int, input[1::2])), fillvalue=0
            )
        )
    )
    total = 0
    file_queue = deque(files)
    free_spaces: list[tuple[int, int]] = []  # (start_index, length)
    pointer = 0

    for file in files:
        pointer += file[1][0]
        if file[1][1] > 0:
            free_spaces.append((pointer, file[1][1]))
        pointer += file[1][1]

    while len(file_queue) > 0:
        file_index, (block_amount, _) = file_queue.pop()
        before = sum([blocks + space for _, (blocks, space) in list(file_queue)])
        fit = next((space for space in free_spaces if space[1] >= block_amount), None)
        if fit is not None and fit[0] < before:
            if fit[1] > block_amount:
                free_spaces[free_spaces.index(fit)] = (
                    fit[0] + block_amount,
                    fit[1] - block_amount,
                )
            else:
                free_spaces.remove(fit)
            for pos in range(fit[0], fit[0] + block_amount):
                total += file_index * pos
            free_spaces.append((before, block_amount))
            free_spaces.sort(key=lambda x: x[0])
        else:
            for pos in range(before, before + block_amount):
                total += pos * file_index
    return total

In [ ]:
solve(parse_input("input"))